<a href="https://colab.research.google.com/github/seunghuilee91/Deeplearning/blob/master/LDA%2C_Latent_Dirichlet_Allocation(%EC%9E%A0%EC%9E%AC%EB%94%94%EB%A6%AC%ED%81%B4%EB%A0%88%ED%95%A0%EB%8B%B9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
import sys
sys.path.append("/content/gdrive/My Drive/Colab Notebooks")

# Latent Dirichlet Allocation, LDA


---


- 말뭉치로부터 토픽을 추출하는 토픽모델링(Topic Modeling) 

- LDA는 토픽모델링 중에서도 가장 큰 영향력!

 - 텍스트 데이터 소프트 클러스터링 

 - 텍스트 구조를 베이지안 네트워크 형태로 표현함 

  - 베이지안 모델이면서 프라이어(priors)를 가짐

- 활용 분야 : 주제 분류나 문서 간 유사도 계산 (검색 엔진, 고객 민원 시스템 등)











### 토픽 모델링이란?
기계 학습 및 자연언어 처리 분야에서 토픽 모델(Topic model)이란 문서 집합의 추상적인 "주제"를 발견하기 위한 통계적 모델 중 하나로, 텍스트 본문의 숨겨진 의미구조를 발견하기 위해 사용되는 텍스트 마이닝 기법 중 하나이다.

## 모델 개요
LDA란 주어진 문서에 대하여 각 문서에 어떤 주제들이 존재하는지에 대한 확률모형입니다. LDA는 토픽별 단어의 분포, 문서별 토픽의 분포를 모두 추정해 냅니다. LDA의 개략적인 도식은 다음과 같습니다.

<img src="https://i.imgur.com/r5e5qvs.png">

우선 LDA는 특정 토픽에 특정 단어가 나타날 확률을 내어 줍니다. 예컨대 위 그림에서 노란색 토픽엔 gene이라는 단어가 등장할 확률이 0.04, dna는 0.02, genetic은 0.01입니다. 이 노란색 토픽은 대략 ‘유전자’ 관련 주제라는 걸 알 수 있네요.

이번엔 문서를 보겠습니다. 주어진 문서를 보면 파란색, 빨간색 토픽에 해당하는 단어보다는 노란색 토픽에 해당하는 단어들이 많네요. 따라서 위 문서의 메인 주제는 노란색 토픽(유전자 관련)일 가능성이 큽니다. 이렇듯 문서의 토픽 비중 또한 LDA의 산출 결과물입니다.

우선 글감 내지 주제를 정해야 합니다. 이후 실제 글을 작성할 때는 어떤 단어를 써야할지 결정합니다. LDA도 마찬가지입니다. 우선 말뭉치로부터 얻은 토픽 분포로부터 토픽을 뽑습니다. 이후 해당 토픽에 해당하는 단어들을 뽑습니다. 이것이 LDA가 가정하는 문서 생성 과정입니다.

이제 반대 방향으로 생각해보겠습니다. 현재 문서에 등장한 단어들은 어떤 토픽에서 뽑힌 단어들일까요? 이건 명시적으로 알기는 어렵습니다. 말뭉치에 등장하는 단어들 각각에 꼬리표가 달려있는 건 아니니까요.

그런데 LDA는 이렇게 말뭉치 이면에 존재하는 정보를 추론해낼 수 있습니다. LDA에 잠재(Latent)라는 이름이 붙은 이유입니다. LDA의 학습은 바로 이러한 **잠재정보를 알아내는 과정**입니다.



## 모델 아키텍처



<img src="https://4four.us/assets/post_image/2010-11-07-latent-dirichlet-allocation-simply-1.png">

> α와 β는 코퍼스 단위로 정해지는 값 <br>
> N과  는 문서 단위로 정해지는 값
[N은 문서의 길이, θ는 해당 문서에서 각 주제의 가중치]

<br>
<br>

**LDA가 가정하는 문서생성과정**
<img src="https://4four.us/assets/post_image/2010-11-07-latent-dirichlet-allocation-simply-2.png">
<BR>

정리하면, 어떤 문서에 대해 파라미터 θ(주제 벡터)가 있고, 앞에서부터 단어를 하나씩 채울 때마다 θ로부터 하나의 주제를 선택하고, 다시 그 주제로부터 단어를 선택하는 방식으로 문서 생성 과정을 모델링 함 

이런 Generative Model을 써서 우리가 새로운 글을 쓰려는 건 당연히 아니다. 하지만, 이런 방법으로 문서 내용을 성공적으로 모델링, 즉 표현할 수 있다면 이미 존재하는 문서의 파라미터 θ를 찾아내는 것도 가능할 것이다. 글 d1과 d2가 있을 때, 주제는 비슷하더라도 각 문서에 등장하는 단어의 종류나 빈도는 다를 수 있기 때문에 단순한 키워드 기반의 모델로는 유사도를 계산하거나 주제 분류를 하는 데에 한계가 있다. 그러나 이미 보유한 많은 텍스트에 기초에 α와 β를 알아 두고, **개별 문서의 θ를 계산할 수 있다면, 이 θ를 가지고 유사도 계산이나 분류 작업을 훨씬 쉽고도 정확하게 해낼 수 있다**.


<br>


**LDA 이름의 의미**

- Latent:

> 사전적인 의미는 “잠재적인, 숨어 있는”. 위에서 설명한 과정에서 우리가 직접 관찰할 수 있는 것은 문서 내용뿐이다. **α, β, θ, z는 모두 감춰진 파라미터**이다.

- Dirichlet: 

> 19세기 독일 수학자의 이름. 디리클레 분포(Dirichlet Distribution)가 그의 이름을 따서 지어졌다고 한다. 제일 위의 코드를 보면 **θ를 결정할 때 α를 파라미터로 디리클레 분포**을 사용한다.

- Allocation: 

> 말 그대로 ‘할당’. 각 단어를 결정할 때, **θ에 대한 다항 분포(Multinomial Distribution)로 주제를 ‘할당**’한 뒤 그 주제로부터 단어를 뽑는다. 

## 잠재 디리클레 할당(LDA) 실습

In [0]:
# 1) 데이터 불러오기 
import pandas as pd
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/franciscadias/data/master/abcnews-date-text.csv", filename="abcnews-date-text.csv")
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)

In [0]:
print(len(data))

In [0]:
# 해당 데이터는 약 100만개의 샘플을 갖고 있습니다. 상위 5개의 샘플만 출력해봅시다.
print(data.head(5))

In [0]:
text = data[['headline_text']]
text.head(5)

In [0]:
# 2) 데이터 전처리
nltk.download('punkt')
import nltk
text['headline_text'] = text.apply(lambda row: nltk.word_tokenize(row['headline_text']), axis=1)

In [0]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
text['headline_text'] = text['headline_text'].apply(lambda x: [word for word in x if word not in (stop)])

In [0]:
print(text.head(5))

In [0]:
#  표제어 추출을 수행합니다. 표제어 추출로 3인칭 단수 표현을 1인칭으로 바꾸고, 과거 현재형 동사를 현재형으로 바꿉니다
nltk.download('wordnet')


In [0]:

from nltk.stem import WordNetLemmatizer
text['headline_text'] = text['headline_text'].apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])
print(text.head(5))

In [0]:
print(text.head(5))

In [0]:
tokenized_doc = text['headline_text'].apply(lambda x: [word for word in x if len(word) > 3])
print(tokenized_doc[:5])

# 이제 길이가 3이하인 단어에 대해서 제거하는 작업을 수행합니다.

In [0]:
# 3) TF-IDF 행렬 만들기
# 기본적으로 토큰화가 되어있지 않은 텍스트 데이터를 입력으로 사용합니다. 이를 사용하기 위해 다시 토큰화 작업을 역으로 취소하는 역토큰화(Detokenization)작업을 수행해보겠습니다.
detokenized_doc = []
for i in range(len(text)):
    t = ' '.join(tokenized_doc[i])
    detokenized_doc.append(t)

text['headline_text'] = detokenized_doc # 다시 text['headline_text']에 재저장

In [0]:
text['headline_text'][:5]

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words='english', 
max_features= 1000) # 상위 1,000개의 단어를 보존 
X = vectorizer.fit_transform(text['headline_text'])
X.shape # TF-IDF 행렬의 크기 확인

In [0]:
# 4) 토픽 모델링
from sklearn.decomposition import LatentDirichletAllocation
lda_model=LatentDirichletAllocation(n_components=10,learning_method='online',random_state=777,max_iter=1)

In [0]:
lda_top=lda_model.fit_transform(X)

In [0]:
print(lda_model.components_)
print(lda_model.components_.shape) 

[기타코드 - 참고] 실패함.

In [0]:
# 사이킷런에서는 Twenty Newsgroups이라고 불리는 20개의 다른 주제를 가진 뉴스 데이터를 제공합니다. 
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
len(documents)

In [0]:
# 훈련에 사용할 뉴스는 총 11,314개입니다. 이 중 첫번째 훈련용 뉴스를 출력해보겠습니다.
documents[1]

In [0]:
# 사이킷런이 제공하는 뉴스 데이터에서 target_name에는 본래 이 뉴스 데이터가 어떤 20개의 카테고리를 갖고있었는지가 저장되어져 있습니다. 이를 출력해보겠습니다.
print(dataset.target_names)

In [0]:
# 시작하기 앞서, 텍스트 데이터에 대해서 가능한한 정제 과정을 거쳐야만 합니다. 기본적인 아이디어는 알파벳을 제외한 구두점, 숫자, 특수 문자를 제거하는 것입니다. 이는 텍스트 전처리 챕터에서 정제 기법으로 배웠던 정규 표현식을 통해서 해결할 수 있습니다. 또한 짧은 단어는 유용한 정보를 담고있지 않다고 가정하고, 길이가 짧은 단어도 제거합니다. 그리고 마지막으로 모든 알파벳을 소문자로 바꿔서 단어의 개수를 줄이는 작업을 합니다.
news_df = pd.DataFrame({'document':documents})
# 특수 문자 제거
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
# 길이가 3이하인 단어는 제거 (길이가 짧은 단어 제거)
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
# 전체 단어에 대한 소문자 변환
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: x.lower())

In [0]:
news_df['clean_doc'][1]

# 우선 특수문자가 제거되었으며, if나 you와 같은 길이가 3이하인 단어가 제거된 것을 확인할 수 있습니다. 뿐만 아니라 대문자가 전부 소문자로 바뀌었습니다. 

In [0]:
pip install NLTK

In [0]:
#  이제 뉴스 데이터에서 불용어를 제거합니다. 불용어를 제거하기 위해서 토큰화를 우선 수행합니다. 토큰화와 불용어 제거를 순차적으로 진행합니다.

from nltk.corpus import stopwords
stop_words = stopwords.words('english') # NLTK로부터 불용어를 받아옵니다.
tokenized_doc = news_df['clean_doc'].apply(lambda x: x.split()) # 토큰화
tokenized_doc = tokenized_doc.apply(lambda x: [item for item in x if item not in stop_words])
# 불용어를 제거합니다.


In [0]:
# 훈련용 뉴스를 5개만 출력해보겠습니다.
tokenized_doc[:5]

In [0]:
#  단어의 빈도수를 기록해보겠습니다. 여기서는 각 단어를 (word_id, word_frequency)

from gensim import corpora
dictionary = corpora.Dictionary(tokenized_doc)
corpus = [dictionary.doc2bow(text) for text in tokenized_doc]
print(corpus[1]) # 수행된 결과에서 두번째 뉴스 출력. 첫번째 문서의 인덱스는 0

# (66, 2) 66번째 할당된 단어가 두번 등장함

In [0]:
print(dictionary[66])

In [0]:
len(dictionary)
#  총 학습된 단어의 개수

In [0]:
# 2) LDA 모델 훈련시키기
# 기존의 뉴스 데이터가 총 20개의 카테고리를 가지고 있었으므로 토픽의 개수를 20으로 하여 LDA 모델을 학습시켜보도록 하겠습니다.

import gensim
NUM_TOPICS = 20              #20개의 토픽, k=20
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)  # passes는 알고리즘의 동작 횟수
topics = ldamodel.print_topics(num_words=4)    # 총 4개의 단어만 출력
for topic in topics:
    print(topic)

In [0]:
print(ldamodel.print_topics())  #10개의 단어를 출력

In [0]:
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    print(i,'번째 문서의 topic 비율은',topic_list)

In [0]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.per_word_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [0]:
topictable = make_topictable_per_doc(ldamodel, corpus)
topictable = topictable.reset_index() # 문서 번호을 의미하는 열(column)로 사용하기 위해서 인덱스 열을 하나 더 만든다.
topictable.columns = ['문서 번호', '가장 비중이 높은 토픽', '가장 높은 토픽의 비중', '각 토픽의 비중']
topictable[:10]

reference

https://wikidocs.net/30708